In [1]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from Dataset.Economy_Dataset import Economy_Dataset
from Dataset.RNN_Transaction_Dataset import RNN_Transaction_Dataset
from Dataset.NODE_Transaction_Dataset import NODE_Transaction_Dataset

from Model.LSTM import LSTM

In [2]:
input_size = 5
hidden_size = 256
output_size = 1

lr = 1e-3
num_epochs = 1

In [3]:
transaction_df = pd.read_excel('../데이터/Transaction/transaction_final.xlsx', index_col=0)
economy_df = pd.read_excel('../데이터/Economy/economy_all.xlsx')
economy_df = economy_df['국고채금리']

In [4]:
trainsaction_train_size = int(len(transaction_df)*0.7)
trainsaction_val_size = int(len(transaction_df)*0.3)

transaction_train_dataset = RNN_Transaction_Dataset(transaction_df[:trainsaction_train_size])
transaction_train_loader = DataLoader(transaction_train_dataset, batch_size=2)
transaction_val_dataset = RNN_Transaction_Dataset(transaction_df[trainsaction_train_size:])
transaction_val_loader = DataLoader(transaction_val_dataset, batch_size=2)

economy_train_size = int(len(economy_df)*0.7)
economy_val_size = int(len(economy_df)*0.3)

economy_train_dataset = Economy_Dataset(economy_df[:economy_train_size])
economy_train_loader = DataLoader(economy_train_dataset, batch_size=2)
economy_val_dataset = Economy_Dataset(economy_df[economy_train_size:])
economy_val_loader = DataLoader(economy_val_dataset, batch_size=2)

In [6]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

### 경제 모델

In [16]:
best_val_loss = float('inf') 
for epoch in range(num_epochs + 1):
    model.train()
    for batch_idx, samples in enumerate(economy_train_loader):
        economy_x_train, economy_y_train = samples

        prediction, hidden = model(economy_x_train)
        cost = criterion(prediction, economy_y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, samples in enumerate(economy_val_loader):
            economy_x_val, economy_y_val = samples

            prediction, hidden = model(economy_x_val)
            loss = criterion(prediction, economy_y_val)
            val_loss += loss.item()

    val_loss /= len(economy_val_loader)
    print(f'Epoch {epoch}/{num_epochs}, Training Loss: {cost.item()}, Validation Loss: {val_loss}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '../데이터/Checkpoint/best_rnn_economy_model.pth')

Epoch 0/1, Training Loss: 0.002730900188907981, Validation Loss: 0.3143093644015106
Epoch 1/1, Training Loss: 0.0027428490575402975, Validation Loss: 0.31638226120186774


### 부동산 모델

In [7]:
best_val_loss = float('inf') 
for epoch in range(num_epochs + 1):
    model.train()
    for batch_idx, samples in enumerate(transaction_train_loader):
        dong_x_train, dong_y_train = samples

        prediction, hidden = model(dong_x_train)
        cost = criterion(prediction, dong_y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, samples in enumerate(transaction_val_loader):
            x_val, y_val = samples

            prediction, hidden = model(x_val)
            loss = criterion(prediction, y_val)
            val_loss += loss.item()

    val_loss /= len(transaction_val_loader)
    print(f'Epoch {epoch}/{num_epochs}, Training Loss: {cost.item()}, Validation Loss: {val_loss}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '../데이터/Checkpoint/best_rnn_transaction_model.pth')

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([2, 1])) that is different to the input size (torch.Size([1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0/1, Training Loss: 21082.39453125, Validation Loss: 5203322.37286441
Epoch 1/1, Training Loss: 26048.66015625, Validation Loss: 5149766.595957344


### NODE 부동산 돌리기

In [ ]:
# 데이터 다운로드
transaction_df = pd.read_excel('../데이터/Transaction/transaction_final.xlsx', index_col=0)

train_dataset = NODE_Transaction_Dataset(transaction_df)
train_loader = DataLoader(train_dataset, batch_size=2)

In [9]:
for x,y,z,w in train_loader:
  print("X 크기 : {}".format(x.shape))
  print("Y 크기 : {}".format(y.shape))
  print("Z 크기 : {}".format(z.shape))
  print("W 크기 : {}".format(w.shape))
  break

X 크기 : torch.Size([2, 5])
Y 크기 : torch.Size([2, 5])
Z 크기 : torch.Size([2, 1])
W 크기 : torch.Size([2, 1])


In [ ]:
# 하이퍼파라미터
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
best_val_loss = float('inf') 
for epoch in range(num_epochs + 1):
    model.train()
    for batch_idx, samples in enumerate(transaction_train_loader):
        dong_x_train, dong_y_train = samples

        prediction, hidden = model(dong_x_train)
        cost = criterion(prediction, dong_y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
    
    model.eval()
    val_loss = 0.0
    with torch.no_grad():
        for batch_idx, samples in enumerate(transaction_val_loader):
            x_val, y_val = samples

            prediction, hidden = model(x_val)
            loss = criterion(prediction, y_val)
            val_loss += loss.item()

    val_loss /= len(transaction_val_loader)
    print(f'Epoch {epoch}/{num_epochs}, Training Loss: {cost.item()}, Validation Loss: {val_loss}')
    
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), '../데이터/Checkpoint/best_rnn_transaction_model.pth')